In [ ]:
import momepy
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import shapely

import osmnx as ox

import scripts.consolidate as consolidate
import scripts.snap as snap

import itertools

import numpy as np
from tqdm.notebook import tqdm
import pygeos

import zipfile
import numpy as np
import mapclassify
import copy
from libpysal.weights import Queen, KNN, W

In [ ]:
local_crs = 3414

place = "Singapore"

latlng = (1.29, 103.85)
dist = 30000

study_area = gpd.read_file("./Singapore_studyArea.shp").to_crs(epsg=4326)

# assuming your geodataframe is called `gdf` and the geometry column is called `geometry`
study_area = study_area.geometry.unary_union

In [ ]:
osm_graph= ox.graph.graph_from_polygon(study_area, network_type='drive')
osm_graph = ox.projection.project_graph(osm_graph, to_crs=local_crs)

streets = ox.consolidate_intersections(osm_graph, rebuild_graph=True, tolerance=15, dead_ends=False)

streets = ox.graph_to_gdfs(
    osm_graph,
    nodes=False,
    edges=True,
    node_geometry=False,
    fill_edge_geometry=True
)
streets.head()

In [ ]:
osm_graph= ox.graph.graph_from_polygon(study_area, custom_filter='["railway"~"rail|subway|narrow_gauge|monorail"]')
osm_graph = ox.projection.project_graph(osm_graph, to_crs=local_crs)

osm_graph = ox.consolidate_intersections(osm_graph, rebuild_graph=True, tolerance=15, dead_ends=False)

rails = ox.graph_to_gdfs(
    osm_graph,
    nodes=False,
    edges=True,
    node_geometry=False,
    fill_edge_geometry=True
)
rails.head()

In [ ]:
waters = []

# Get the water bodies as geometries
water_bodies = ox.geometries_from_polygon(study_area, tags={'natural': 'water'}).set_crs("epsg: 4326").to_crs(local_crs)

# Explode the GeoDataFrame into a GeoSeries of polygons and multipolygons
gs = water_bodies.explode()

# Convert each polygon into a MultiLineString
mls = gs.geometry.boundary

# Convert the MultiLineString into a DataFrame
water_bodies = gpd.GeoDataFrame({'geometry': mls})

# Add a column to specify the polygon ID
water_bodies['polygon_id'] = gs.index

print(water_bodies)

for water_body in water_bodies.geometry:
    # Create a LineString object from the coordinates
    waters.append(water_body)

coastlines = ox.geometries_from_polygon(study_area, tags={'natural': 'coastline'}).set_crs("epsg: 4326").to_crs(local_crs)

coastlines = coastlines[coastlines.geom_type == "LineString"].reset_index(drop=True)

for coastline in coastlines.geometry:
    # Create a LineString object from the coordinates
    waters.append(coastline)

# # Create a GeoDataFrame of the water area boundaries
waters = gpd.GeoDataFrame(geometry=waters)
waters = waters[["geometry"]]
waters.head()

In [ ]:
ax = streets.plot(figsize=(16, 16))
rails.plot(ax=ax, color='r')
waters.plot(ax=ax, color='k')

In [ ]:
%%time
barriers = pd.concat([streets.geometry, waters.geometry])

In [ ]:
%%time
unioned = barriers.unary_union

In [ ]:
%%time
polygons = shapely.ops.polygonize(unioned)
enclosures = gpd.array.from_shapely(list(polygons), crs=local_crs)

In [ ]:
%%time
railway_topo = consolidate.topology(rails)

In [ ]:
%%time
closed = snap.close_gaps(railway_topo, tolerance=25)

In [ ]:
%%time
closed_topo = consolidate.topology(gpd.GeoDataFrame(geometry=closed))

In [ ]:
%%time
extended_topo = snap.line_to_line(closed_topo, streets, 25)

In [ ]:
%%time
additional = pd.concat([waters.geometry, extended_topo.geometry])

In [ ]:
%%time
sindex = gpd.GeoSeries(enclosures).sindex
inp, res = sindex.query_bulk(additional.geometry, predicate='intersects')

In [ ]:
%%time
unique = np.unique(res)

In [ ]:
%%time
new = []

for i in tqdm(unique, total=len(unique)):
    poly = enclosures.data[i]  # get enclosure polygon
    crossing = inp[res==i]  # get relevant additional barriers
    buf = pygeos.buffer(poly, 0.01)  # to avoid floating point errors
    crossing_ins = pygeos.intersection(buf, additional.values.data[crossing])  # keeping only parts of additional barriers within polygon
    union = pygeos.union_all(np.append(crossing_ins, pygeos.boundary(poly)))  # union
    polygons = np.array(list(shapely.ops.polygonize(_pygeos_to_shapely(union))))  # shapely.ops.polygonize
    within = pygeos.covered_by(pygeos.from_shapely(polygons), buf)  # keep only those within original polygon

In [ ]:
%%time
final_enclosures = gpd.GeoSeries(enclosures).drop(unique).append(gpd.GeoSeries(new))

In [ ]:
final_enclosures.shape

In [ ]:
%%time
gpd.GeoDataFrame(geometry=final_enclosures, crs=local_crs).to_parquet('./out/singapore/enclosures.pq')

In [ ]:
auth = gpd.read_file("./Singapore/Singapore_constituencies.shp").to_crs(local_crs)
auth["index"] = range(len(auth))

In [ ]:
buildings = ox.geometries.geometries_from_polygon(study_area, tags={'building':True})
buildings = buildings[buildings.geom_type == "Polygon"].reset_index(drop=True)
buildings = buildings[["geometry"]].to_crs(local_crs)

In [ ]:
auth_buildings = gpd.sjoin(auth, buildings, op='intersects')
count = auth_buildings.groupby('index').size()

In [ ]:
auth = auth.merge(count.rename('counts'), left_on='index', right_index=True)

In [ ]:
auth.head()

In [ ]:
%time w_queen = Queen.from_dataframe(auth)

In [ ]:
%time w_k1 = KNN.from_dataframe(auth, k=1)

In [ ]:
neighbors = copy.deepcopy(w_queen.neighbors)
for i in w_queen.islands:
    j = w_k1.neighbors[i][0]
    neighbors[i] = [j]
    neighbors[j].append(i)
w = W(neighbors)